<a href="https://colab.research.google.com/github/echung2/echung2/blob/master/%ED%95%9C%EA%B5%AD%ED%98%84%EB%8C%80%EB%AC%B8%ED%95%99%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EC%97%B0%EA%B5%AC_8%EC%A3%BC%EC%B0%A8_%EA%B3%B5%EA%B8%B0%EC%96%B4_word2vec_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 공기어 분석

### 0. 자료 준비

In [ ]:
# 나눔고딕
!apt-get update -qq
!apt-get install fonts-nanum* -qq
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 패키지 설치
!pip install -U gensim kiwipiepy flashtext scikit-learn delayed -q

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from kiwipiepy import Kiwi, Option
kiwi = Kiwi()
kiwi.prepare()

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
plt.rc('font', family='NanumBarunGothic')

import itertools
from collections import Counter

import regex #확장된 정규표현식. 일반 정규표현식은 import re

from flashtext import KeywordProcessor
kp = KeywordProcessor()

from gensim.test.utils import common_texts
from gensim.models import Word2Vec

from wordcloud import WordCloud
wordcloud = WordCloud()

In [ ]:
# 이인직 소설 자료 다운로드
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1AY763FcPXN_iBo_sVMXHugQ8UHFvb_nN' -O lee.xlsx

In [ ]:
df = pd.read_excel('lee.xlsx')
df

### 1. 형태소 분석 및 전처리

##### 사용자 사전 등록

In [ ]:
kiwi.analyze('혈의누의 옥련은 운다. 훌쩍이다')

In [ ]:
# 사용자 사전 활용
# kiwi.load_user_dictionary("dic.txt")
# kiwi.prepare()

In [ ]:
kiwi.analyze('옥련은 혈의누의 주인공이다.')

##### 형태소 분석
품사 참고 : https://bab2min.github.io/kiwipiepy/v0.9.2/kr/#_7

In [ ]:
# 몇가지 품사 제외한 모든 품사 추출
def tokenize(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '') # 동사/형용사에는 '다'를 붙여줌
    # return [word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S')] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

In [ ]:
# 몇가지 품사 제외한 모든 품사 추출 + 품사 태그 포함
def tokenize_tag(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '')+ '/'+ tag # 동사/형용사에는 '다'를 붙여줌 + / 품사
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S')] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거

In [ ]:
# 특정 품사만 추출
def tokenize_part(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word
            for word, tag, _, _ in res
            if tag.startswith('NN')] # 'NN'으로 시작하는 품사만 추출 = 명사만 추출

In [ ]:
df['paragraph'][0]

In [ ]:
tokenize_tag(df['paragraph'][0])

In [ ]:
df['tokens'] = df['paragraph'].progress_map(lambda x:tokenize_tag(x))

In [ ]:
df['tokens']

##### 불용어 제거

In [ ]:
# 상위 n개 단어 확인
token_list = list(itertools.chain(*df['tokens'].tolist()))
cnt = Counter(token_list)
cnt.most_common(30) # 상위 20개

In [ ]:
 # 불용어 리스트
stopwords = set(['이다/VCP','하다/VV','하다/VX','위하다/VV','되다/VV','있다/VV', '있다/VX','없다/VA','않다/VX','있다/VV','아니하다/VX'])

In [ ]:
# 불용어 제거
df['tokens'] = df['tokens'].map(lambda x:[w for w in x if not w in set(stopwords)])

In [ ]:
# 유효한 1음절은 살리고 나머지는 제거
hangul_1_except = regex.compile(r'^(?!일/NNG|돈/NNG|꿈/NNG|집/NNG)\p{Hangul}{1}/\w+$') # 파이프(|)로 구분해 입력. 맨마지막 단어에는 파이프 입력하지 말것.

In [ ]:
df['tokens'] = df['tokens'].progress_map(lambda x:[w for w in x if not hangul_1_except.match(w)]) #매치되는 것을 제거한다.

In [ ]:
df['tokens']

##### 동의어 처리

In [ ]:
# flashtext용 동의어 딕셔너리
# 통일단어 : 바꿀단어
#'startup':['start up','start ups','start-up','start-ups','start_up','start_ups'],

synonym_dict = {
'옥련':['옥련이'] # 부인?

}
kp.add_keywords_from_dict(synonym_dict)

In [ ]:
# 동의어 처리 실행
df['tokens'] = [[kp.replace_keywords(x) for x in w] for w in tqdm(df['tokens'])]

##### 기타 전처리 (유효한 행만 남긱기)

In [ ]:
# 특정 형태소가 들어간 행만 남기기
query_list = ['울다/VV','훌쩍이다/VV'] # OR 조건
# df['tokens'].map(lambda x:query in x)
df[df['tokens'].map(lambda x:any(query in x for query in query_list))]

In [ ]:
df['tokens'].map(lambda x:len(x))

In [ ]:
df['tokens'].map(lambda x:len(x)).describe()

In [ ]:
df['tokens'].map(lambda x:len(x)).hist()

In [ ]:
# token 개수가 N개 이상인 행만 살리기
df = df[df['tokens'].map(lambda x:len(x)>=3)] #3개 이상
df

In [ ]:
# paragraph 열 중복행이 있으면 제거 
df = df.drop_duplicates(subset=['paragraph'])

In [ ]:
# reset index
df = df.reset_index(drop=True)

### 2. 키워드 추출 / 단어 네트워크

In [ ]:
# 각 작품별로 따로 분석하고 싶다면 아래 코드를 실행하고 분석 (# 제거하고 실행)
# df = df[df['title']=='혈의 누/현대어 해석'] #혈의 누

##### Term Frequency (단순 빈도수)

In [ ]:
tf_vectorizer = CountVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=['NNG'],
                             token_pattern = r'(?u)\b\w\w+(?:\/)?\w+\b',
                             min_df=50, # 최소 N개 문서(단락)에서 등장해야 함
                             ngram_range=(1,2) #bigram까지
                             )

In [ ]:
tf_vector = tf_vectorizer.fit_transform(df['tokens'].astype(str))

In [ ]:
# 빈도수 내림차순으로 정렬
tf_scores = tf_vector.toarray().sum(axis=0)
tf_idx = np.argsort(-tf_scores)
tf_scores = tf_scores[tf_idx]
tf_vocab = np.array(tf_vectorizer.get_feature_names())[tf_idx]

In [ ]:
# 상위 50개 단어
print(list(zip(tf_vocab, tf_scores))[:50])

In [ ]:
# 워드 클라우드 (참고용)
keywords = dict(zip(tf_vocab, tf_scores))

# wordcloud = wordcloud.generate_from_text(texts)
wordcloud = wordcloud.generate_from_frequencies(keywords)
wordcloud = WordCloud(
    font_path = font_path,
    width = 800,
    height = 800
)
wordcloud = wordcloud.generate_from_frequencies(keywords)
array = wordcloud.to_array()
fig = plt.figure(figsize=(5, 5))
plt.imshow(array, interpolation="bilinear")
plt.show()

##### TF-IDF(빈도수 * 역문서 빈도수)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=['NNG'],
                             token_pattern = r'(?u)\b\w\w+(?:\/)?\w+\b',
                             min_df=50, 
                             ngram_range=(1,2), #bigram 
                             smooth_idf=True)

In [ ]:
tfidf_vector = tfidf_vectorizer.fit_transform(df['tokens'].astype(str))

In [ ]:
tfidf_scores = tfidf_vector.toarray().sum(axis=0)
tfidf_idx = np.argsort(-tfidf_scores)
tfidf_scores = tfidf_scores[tfidf_idx]
tfidf_vocab = np.array(tfidf_vectorizer.get_feature_names())[tfidf_idx]

In [ ]:
print(list(zip(tfidf_vocab, tfidf_scores))[:50]) #상위 50개 단어

##### 단어 빈도수 테이블 정리

In [ ]:
###TF, TF-IDF 단어 테이블 정리###
list(zip(tf_vocab, tf_scores,tfidf_vocab,tfidf_scores))[:100] #상위 100개
tf_tfidf_vocab = pd.DataFrame(list(zip(tf_vocab, tf_scores,tfidf_vocab,tfidf_scores)),
                              columns=['TF 단어','TF','TFIDF 단어','TFIDF'])

In [ ]:
tf_tfidf_vocab

In [ ]:
tf_tfidf_vocab.to_excel('이인직 키워드 빈도.xlsx') # 엑셀파일로 저장

##### 코사인 유사도 기반의 단어-단어 행렬

In [ ]:
tfidf_term_term_mat = cosine_similarity(tfidf_vector.T)
tfidf_term_term_mat = pd.DataFrame(tfidf_term_term_mat,index=tfidf_vectorizer.vocabulary_,columns=tfidf_vectorizer.vocabulary_)

In [ ]:
tfidf_term_term_mat

In [ ]:
# tf-idf 기준 상위 100개 단어만# tf-idf 기준 상위 100개 단어만
tfidf_term_term_mat_100 = tfidf_term_term_mat[tfidf_term_term_mat.keys().isin(tfidf_vocab[:100])]
tfidf_term_term_mat_100 = tfidf_term_term_mat_100[tfidf_term_term_mat_100.columns.intersection(tfidf_vocab[:100])]
tfidf_term_term_mat_100

In [ ]:
# csv로 저장
tfidf_term_term_mat_100.iloc[:100,:100].to_csv('단어 매트릭스.csv')

## word2vec

### 모델링

In [ ]:
sentences = df['tokens'].to_list()

In [ ]:
model = Word2Vec(sentences=sentences, vector_size=200, window=3, min_count=10) # min_count = 10이면 10회 이상 등장한 단어만 학습

In [ ]:
# 유의어 뽑기
model.wv.most_similar('소리/NNG', topn=20) 

In [ ]:
# 유의어 뽑기
model.wv.most_similar('울다/VV', topn=20) 

### 시각화

In [ ]:
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
import gensim 
import gensim.models as g

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
tsne =TSNE(n_components=2)

In [ ]:
vocab = list(model.wv.key_to_index)
X = model.wv[vocab]

In [ ]:
X_tsne = tsne.fit_transform(X)

In [ ]:
%matplotlib inline  
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# 체크해보면 폰트 개수가 늘어났다
sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

In [ ]:
# 100개의 단어에 대해서만 시각화
X_tsne = tsne.fit_transform(X[:100,:])
# X_tsne = tsne.fit_transform(X)

w2v_df = pd.DataFrame(X_tsne, index=vocab[:100], columns=['x', 'y'])
w2v_df.shape
w2v_df.head(10)

In [ ]:
# path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
# font_name = fm.FontProperties(fname=path, size=8).get_name()
# print(font_name)
# plt.rc('font', family=font_name)

In [ ]:
# !python --version
# def current_font():
#   print(f"설정 폰트 글꼴: {plt.rcParams['font.family']}, 설정 폰트 사이즈: {plt.rcParams['font.size']}")  # 파이썬 3.6 이상 사용가능하다
        
# current_font()

In [ ]:
plt.rc('font', family='NanumBarunGothic')
fig = plt.figure()
fig.set_size_inches(40, 20)
ax = fig.add_subplot(1, 1, 1)

ax.scatter(w2v_df['x'], w2v_df['y'])

for word, pos in w2v_df.iterrows():
    ax.annotate(word, pos, fontsize=20)
plt.show()